**Proyecto clasificación**

* Paula Rojas
* Diego Sandoval




**Interes y Objetivo del proyecto**


El interés del proyecto es clasificar las hojas de café con respecto a su sanidad o en el segundo caso identificar si las plantas contienen una de las 2 plagas: Roya y Minador.
La roya es un hongo fitoparásito que es exclusivo de los cafetales. Es considerada una de las enfermedades más catastróficas de toda la historia que puede afectar a los cultivos y está dentro de las que más pérdidas ha ocasionado en los últimos 100 años.
Los minadores de las hojas son larvas del insecto díptero perteneciente al género Liriomyza. Forman galerías en las hojas y llegan a ser un grave problema en varios cultivos hortícolas y ornamentales.
Identificar las características de las hojas de café para aquellos que quieren hacer uso de ellas, predecir qué tan sana es la planta o si contiene plagas y prevenir la pérdida de cultivos.


**Población Afectada**
 
Con este proyecto se deseea ayudar a los cultivadores a poder identificar si las hojas de café de sus cultivos se encuentran afectadas o no por alguna de las plagas mencionadas, y así tomar las medidas de cuidado respectivas para poder controlar dichas plagas y perder el menor número de cultivo de café.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dropout, Flatten, Dense, Activation
from tensorflow.python.keras.layers import  Convolution2D, MaxPooling2D
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.optimizer_v2.adam import Adam


K.clear_session()

data_entrenamiento = '/content/drive/MyDrive/Cafe/entrenamiento'
data_validacion = '/content/drive/MyDrive/Cafe/validacion'

"""
Parameters
"""
epocas= 12
longitud, altura = 100, 100 
batch_size = 20
pasos = 20
validation_steps = 4
filtrosConv1 = 32
filtrosConv2 = 64
tamano_filtro1 = (3, 3)
tamano_filtro2 = (2, 2)
tamano_pool = (2, 2)
clases = 3
lr = 0.0005

#Preparamos nuestras imagenes

entrenamiento_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2, 
    zoom_range=0.2,  
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(
    rescale=1. / 255
)  

entrenamiento_generador = entrenamiento_datagen.flow_from_directory(    
    data_entrenamiento,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical'
)

validacion_generador = test_datagen.flow_from_directory(    
    data_validacion,
    target_size=(altura, longitud),
    batch_size=batch_size,
    class_mode='categorical'
)

#Crear CNN

cnn = Sequential()

cnn.add(Convolution2D(filtrosConv1, tamano_filtro1, padding ="same", input_shape=(longitud, altura, 3), activation='relu'))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Convolution2D(filtrosConv2, tamano_filtro2, padding ="same"))
cnn.add(MaxPooling2D(pool_size=tamano_pool))

cnn.add(Flatten())
cnn.add(Dense(256, activation='relu'))
cnn.add(Dropout(0.5))
cnn.add(Dense(clases, activation='softmax'))

adam = Adam()

cnn.compile(
    loss='categorical_crossentropy',
    optimizer=adam,
    metrics=['accuracy']
)

cnn.fit(
    entrenamiento_generador,  
    steps_per_epoch=pasos,
    epochs=epocas,
    validation_data=validacion_generador,   
    validation_steps=validation_steps
)     

target_dir = '/content/drive/MyDrive/Cafe/modelo/'  

if not os.path.exists(target_dir):   
  os.mkdir(target_dir)               
cnn.save('/content/drive/MyDrive/Cafe/modelo/modelo.h5')
cnn.save_weights('/content/drive/MyDrive/Cafe/modelo/pesos.h5')

Found 147 images belonging to 3 classes.
Found 120 images belonging to 3 classes.
Epoch 1/12
20/20 [==============================] - 36s 2s/step - loss: 2.8129 - accuracy: 0.3797 - val_loss: 1.2129 - val_accuracy: 0.3250
Epoch 2/12
20/20 [==============================] - 33s 2s/step - loss: 1.0676 - accuracy: 0.4515 - val_loss: 1.1355 - val_accuracy: 0.3125
Epoch 3/12
20/20 [==============================] - 35s 2s/step - loss: 0.9121 - accuracy: 0.5668 - val_loss: 1.3155 - val_accuracy: 0.3500
Epoch 4/12
20/20 [==============================] - 34s 2s/step - loss: 0.8438 - accuracy: 0.5956 - val_loss: 1.1578 - val_accuracy: 0.3500
Epoch 5/12
20/20 [==============================] - 38s 2s/step - loss: 0.7508 - accuracy: 0.6310 - val_loss: 1.3669 - val_accuracy: 0.4250
Epoch 6/12
20/20 [==============================] - 35s 2s/step - loss: 0.7266 - accuracy: 0.6759 - val_loss: 1.4558 - val_accuracy: 0.3500
Epoch 7/12
20/20 [==============================] - 34s 2s/step - loss: 0.6257

In [20]:
import numpy as np
from keras.preprocessing.image import load_img, img_to_array
from keras.models import load_model

longitud, altura = 100, 100
modelo = '/content/drive/MyDrive/Cafe/modelo/modelo.h5'
pesos_modelo = '/content/drive/MyDrive/Cafe/modelo/pesos.h5'
cnn = load_model(modelo)
cnn.load_weights(pesos_modelo)

def predict(file):
  x = load_img(file, target_size=(longitud, altura))
  x = img_to_array(x)
  x = np.expand_dims(x, axis=0)
  array = cnn.predict(x) 
  result = array[0]
  answer = np.argmax(result)
  if answer == 0:
    print("pred: Minador")
  elif answer == 1:
    print("pred: Roya")
  elif answer == 2:
    print("pred: Sano")

  return answer

print("****Imágenes de la plaga minador****")
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/101.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/102.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/103.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/104.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/105.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/106.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/107.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/108.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/109.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/PlagaMinador/110.jpg')

print("****Imágenes de la roya****")
predict('/content/drive/MyDrive/Cafe/prueba/Roya/101.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/102.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/103.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/104.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/105.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/106.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/107.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/108.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/109.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Roya/110.jpg')

print("****Imágenes de hojas sanas****")
predict('/content/drive/MyDrive/Cafe/prueba/Sano/101.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/102.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/103.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/104.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/105.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/106.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/107.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/108.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/109.jpg')
predict('/content/drive/MyDrive/Cafe/prueba/Sano/110.jpg')

****Imágenes de la plaga minador****
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Minador
****Imágenes de la roya****
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Sano
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Minador
****Imágenes de hojas sanas****
pred: Minador
pred: Roya
pred: Minador
pred: Minador
pred: Minador
pred: Minador
pred: Sano
pred: Minador
pred: Minador
pred: Minador


0